In [1]:
# Algorithms Project 1: RSA
# Objective: Implement RSA Encryption and apply it to digital signature

In [2]:
# Import libraries
import secrets as secrets
import random as rand
import hashlib

In [3]:
# Performs Fermat's Primality Test on a given value
# @param potentialPrime The number being tested
# @param iterations The number of iterations, higher takes longer but is more accurate
# @return True if potentialPrime *might* be prime; False if proven composite
def prime_test(potentialPrime, iterations = 5):
    # test for small values
    if potentialPrime <= 4:
        return not ((potentialPrime % 2) == 0)
    
    # perform test iterations
    for _ in range(iterations):
        a = rand.randint(2, potentialPrime - 2)
        if pow(a, potentialPrime - 1, potentialPrime) != 1:
            return False

    # If no test has proven composite, number is probably prime
    return True

In [4]:
# Generates a single prime number with a bit length (>= 128) and (< 256)
# @return Random prime number
def generate_prime():
    while True:
        # generate a random number
        prime = pow(2, 128) + secrets.randbelow(pow(2, 256) - pow(2, 128))

        # Test if probably prime
        if prime_test(prime):
            return prime

In [5]:
# Saves two values to a text file, each value on a separate line
# @param fileName The values will be saved to "fileName.txt"
# @param val1 The value to appear on the first line
# @param val2 The value to appear on the second line
def save_value_pair(fileName, val1, val2):
    # Open a file in write mode ('w')
    with open((fileName + '.txt'), 'w') as file:
        # Save values on separate lines
        file.write(f"{val1}\n")
        file.write(f"{val2}")

    # Confirm success
    print("Saved integer pair to `" + fileName + ".txt`")
    return

In [6]:
# Finds the greatest common denominator using Extended Euclidean Algorithm
# @param a First number to be compared
# @param b Second number to be compared
# @return gcd Greatest common divisor of a and b
# @return x, y Bezout coefficients
def gcd_with_bezout(a, b):
    # base case
    if a == 0:
        return b, 0, 1

    # recursion
    gcd, x1, y1 = gcd_with_bezout(b % a, a)

    # update bezout coefficients
    x = y1 - (b//a) * x1
    y = x1
    
    return gcd, x, y

In [7]:
# Generates two pairs of keys from two prime numbers using Extended Euclidean Algorithm
def generate_RSA_key():
    # Generate pair of primes, save to file
    p = generate_prime()
    q = generate_prime()
    save_value_pair("p_q", p, q)
    
    # Solve for lambda(n)
    temp, _, _ = gcd_with_bezout(p-1, q-1)
    lambda_n = ((p-1) * (q-1)) // temp

    # Solve for public/private keys
    n = p*q
    e = pow(2, 16) + 1
    _, _, d = gcd_with_bezout(lambda_n, e)

    # Save keys
    save_value_pair("e_n", e, n)
    save_value_pair("d_n", d, n)

    # Error if de % lambda(n) != 1
    if (d*e % lambda_n != 1):
        print("Error: [de % lambda(n) != 1]!")
    return

In [8]:
# Sign a message and save to a new file
# @param doc filename of message
# @param key filename of private key
def Signing(doc, key):
    # Initialize variables
    message = ""
    d = ""
    n = ""
    
    # Open files in read mode ('r')
    with open(doc, 'r') as file:
        # Read message
        lines = file.readlines()
        message = lines[0]
    with open(key, 'r') as file:
        # Read message
        lines = file.readlines()
        d = int(lines[0], 10)
        n = int(lines[1], 10)

    # Hash message
    hash = hashlib.sha256(message.encode("utf-8"))
    m = hash.hexdigest()

    # Sign Message
    signature = int(pow(int(m, 16), d, n))
    print("Message:", message)

    # Save to file
    with open(doc + ".signed", 'w') as file:
        # Save values on separate lines
        file.write(message + "\n")
        file.write(f"{signature}")

In [9]:
# Verify that a message comes from a particular person
# @param doc filename of signed message
# @param key filename of public key
def Verification(doc, key):
    print("Verifying signed file \"" + doc + "\"...")
    # Initialize variables
    message = ""
    signature = ""
    e = ""
    n = ""
    
    # Open files in read mode ('r')
    with open(doc, 'r') as file:
        # Read message
        lines = file.readlines()
        message = lines[0].strip()
        signature = int(lines[1], 10)
    with open(key, 'r') as file:
        # Read message
        lines = file.readlines()
        e = int(lines[0], 10)
        n = int(lines[1], 10)

    # Generate hashes
    hashFromSignature = pow(signature, e, n)
    hashFromMessage = int(hashlib.sha256(message.encode("utf-8")).hexdigest(), 16)

    # Compare hashes
    if hashFromMessage == hashFromSignature:
        # Success!
        print("Signature successfully verified!\n")
    else:
        # Failure...
        print("Signature has FAILED verification.\n")

In [10]:
# Edits the message contents of a signed message
def tamper(filename, fakeMessage):
    # Open files in read/write mode ('r+')
    with open(filename, 'r+') as file:
        # Read message
        lines = file.readlines()
        signature = lines[1]

        # Clear file contents
        file.truncate(0)
        
        # Save new message and old signature
        file.write(fakeMessage)
        file.write("\n")
        file.write(signature)

    # Confirm message tampering
    print("Signed message tampered; now reads:", fakeMessage, "\n")

In [11]:
# Main function for Project 1
def CPSC_435_Project1(part, task="", fileName=""):
    if part == 1:
        # Project Part 1
        # Generates two sufficiently large primes and two pairs of keys, all saved in respective files:
        # `p_q.txt` holds both prime numbers (would be deleted in practice)
        # `e_n.txt` holds public encryption key
        # `d_n.txt` holds private decryption key
        generate_RSA_key()
    else:
        # Project Part 2
        # Generate and verify digital signatures with a SHA-256 hash
        if "s" in task:
            # Project Part 2a
            doc = fileName # message to be signed
            key = "d_n.txt" # private key

            # sign document
            Signing(doc, key)
        else:
            # Project Part 2b
            # do verification
            doc = fileName # signed filename
            key = "e_n.txt" # public key
            Verification(doc, key)

    if part == 1:
        print("Keys generated!")
    elif "s" in task:
        print("Message", "\"" + fileName + "\"", "signed!\n")
    return

In [12]:
# Generate a set of keys
CPSC_435_Project1(1)

Saved integer pair to `p_q.txt`
Saved integer pair to `e_n.txt`
Saved integer pair to `d_n.txt`
Keys generated!


In [13]:
# Correct message example

# Sign message
CPSC_435_Project1(2, "s", "message1.txt")

# Verify valid message
CPSC_435_Project1(2, "d", "message1.txt.signed")

Message: We're no strangers to love. You know the rules, and so do I. A full commitment's what I'm thinking of. You wouldn't get this from any other guy.
Message "message1.txt" signed!

Verifying signed file "message1.txt.signed"...
Signature successfully verified!



In [14]:
# Tampered message example

# Sign message
CPSC_435_Project1(2, "s", "message2.txt")

# Tamper with message
fakeMessage = "There once was a ship that put to see and the name of that ship was a Jolly Sardine!"
tamper("message2.txt.signed", fakeMessage)

# Verify tampered message
CPSC_435_Project1(2, "d", "message2.txt.signed")

Message: There once was a ship that put to see and the name of that ship was a Billy O' Tea!
Message "message2.txt" signed!

Signed message tampered; now reads: There once was a ship that put to see and the name of that ship was a Jolly Sardine! 

Verifying signed file "message2.txt.signed"...
Signature has FAILED verification.

